<h2>CIC_TON_IOT_GNN_TRAINING flow__multi_class__n_feats__unsorted</h2>

<h1>Creating Graphs</h1>

In [1]:
!pip install powerlaw

In [2]:
import json

with open("/kaggle/input/cic-ton-iot-gnn/df_properties.json", "r") as f:
    df_properties = json.load(f)

print("📘 Dataset Properties:")
for k, v in df_properties.items():
    print(f"{k}: {v}")


📘 Dataset Properties:
name: cic_ton_iot
length: 5350583
num_benign: 2514059
percentage_of_benign_records: 46.98663678331875
num_attack: 2836524
percentage_of_attack_records: 53.01336321668125
attacks: ['Benign', 'mitm', 'scanning', 'dos', 'ddos', 'injection', 'password', 'backdoor', 'ransomware', 'xss']
weak_columns: ['Pkt Len Max', 'Bwd Seg Size Avg', 'Fwd Seg Size Avg', 'Fwd URG Flags', 'Bwd Pkt Len Max', 'Bwd URG Flags', 'Fwd Pkt Len Mean', 'Pkt Len Mean', 'FIN Flag Cnt', 'Bwd Header Len', 'Flow IAT Min', 'Tot Bwd Pkts', 'Flow Duration', 'Flow IAT Max', 'TotLen Bwd Pkts', 'Bwd Pkt Len Std', 'Active Std', 'Fwd Pkts/b Avg', 'Fwd Byts/b Avg', 'Fwd IAT Mean', 'Fwd Pkt Len Std', 'Pkt Len Var', 'Fwd Blk Rate Avg', 'Subflow Bwd Pkts', 'Pkt Len Std', 'CWE Flag Count', 'Bwd PSH Flags', 'Flow Pkts/s', 'URG Flag Cnt', 'Active Mean', 'Idle Mean', 'Bwd IAT Mean', 'Bwd IAT Tot', 'Flow IAT Mean', 'Fwd IAT Tot', 'PSH Flag Cnt', 'Bwd Pkt Len Mean', 'Pkt Size Avg', 'Fwd Pkt Len Max']


In [3]:
import pickle

with open("/kaggle/input/cic-ton-iot-gnn/labels_names.pkl", "rb") as f:
    labels_names = pickle.load(f)
print(labels_names)
print("\n🏷️ Label Class Mapping:")
# for k, v in labels_names.items():
#     print(f"{k}: {v}")


[{0: 'Benign', 1: 'backdoor', 2: 'ddos', 3: 'dos', 4: 'injection', 5: 'mitm', 6: 'password', 7: 'ransomware', 8: 'scanning', 9: 'xss'}, array(['Benign', 'mitm', 'scanning', 'dos', 'ddos', 'injection',
       'password', 'backdoor', 'ransomware', 'xss'], dtype=object)]

🏷️ Label Class Mapping:


In [4]:
import os
import pickle
import random
import socket
import struct

import sys
import pandas as pd
sys.path.append("/kaggle/input/gnn-nids/pytorch/default/1")

import networkx as nx
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from src.dataset.dataset_info import datasets
from src.graph.graph_construction import create_weightless_window_graph
from src.graph.graph_measures import calculate_graph_measures
from src.graph.centralities import add_centralities, add_centralities_as_node_features
# from local_variables import local_datasets_path
local_datasets_path = "/kaggle/input/"
original_datasets_files_path = "/kaggle/input/cic-ton-iot-gnn"
# /kaggle/input/cic-ids-2017-gnn-2


In [5]:
multi_class = True

use_node_features = True

use_port_in_address = False

generated_ips = False

graph_type = "flow"
# graph_type = "window"
# graph_type = "line"

window_size= 500
# print("1. flow sorted")
# print("2. flow using node features unsorted") 
print("3. flow using node features sorted")
# 4. flow using port numbers sorted

sort_timestamp = True

# k_fold = None
# k_fold = 5

validation_size = 0.1
test_size = 0.1

cn_measures = ["betweenness", "degree", "pagerank", "closeness", "k_truss"]
# cn_measures = ["betweenness", "degree", "closeness"]

network_features = ['src_betweenness', 'dst_betweenness', 'src_degree', 'dst_degree', 'src_pagerank', 'dst_pagerank', 'src_closeness', 'dst_closeness', 'src_k_truss', 'dst_k_truss']
# network_features = ['src_betweenness', 'dst_betweenness', 'src_degree', 'dst_degree', 'src_pagerank', 'dst_pagerank']

3. flow using node features sorted


In [6]:
# name = "cic_ton_iot_5_percent"
name = "cic_ton_iot"
# name = "cic_ids_2017_5_percent"
# name = "cic_ids_2017"
# name = "cic_bot_iot"
# name = "cic_ton_iot_modified"
# name = "nf_ton_iotv2_modified"
# name = "ccd_inid_modified"
# name = "nf_uq_nids_modified"
# name = "edge_iiot"
# name = "nf_cse_cic_ids2018"
# name = "nf_bot_iotv2"
# name = "nf_uq_nids"
# name = "x_iiot"

dataset = datasets[name]

In [7]:
g_type = ""
if graph_type == "flow":
    g_type = "flow"
elif graph_type == "line":
    g_type = f"line_graph_{window_size}"
elif graph_type == "window":
    g_type = f"window_graph_{window_size}"
    
if multi_class:
    g_type += "__multi_class"
    
if use_node_features:
    g_type += "__n_feats"
    
# if k_fold:
#     g_type += f"__{k_fold}_fold"
    
if use_port_in_address:
    g_type += "__ports"
    
if generated_ips:
    g_type += "__generated_ips"
    
if sort_timestamp:
    g_type += "__sorted"
else:
    g_type += "__unsorted"
# ****************** ALTERED    



dataset_path = os.path.join(local_datasets_path,name)
folder_path = os.path.join(dataset_path, g_type)
if name == "cic_ids_2017":
    name_ds = "cic-ids-2017-gnn-2"
    dataset_path = os.path.join(local_datasets_path,name_ds)
    # folder_path = os.path.join(local_datasets_path,name, g_type)
    folder_path= os.path.join("/kaggle/working/",name,g_type)
if name =="cic_ton_iot":
    name_ds = "cic-ton-iot-gnn"
    dataset_path = os.path.join(local_datasets_path,name_ds)
    folder_path= os.path.join("/kaggle/working/",name,g_type)
    
print(folder_path)
print(dataset_path)

/kaggle/working/cic_ton_iot/flow__multi_class__n_feats__sorted
/kaggle/input/cic-ton-iot-gnn


In [8]:
# if name== "cic_ids_2017":
#     df=pd.read_parquet(os.path.join(dataset_path, f"{name}.parquet"))
# else:
df = pd.read_parquet(os.path.join(dataset_path, f"{name}.parquet"))

In [9]:
df.head()

,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,...,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label,Attack,Class
0,179528,177.30.87.144,0.0,192.168.1.1,0.0,0.0,2019-04-25 17:18:52+00:00,47814343.0,5.0,0.0,...,0.000000e+00,1038036.0,1038036.0,5.187256e+14,8.984590e+14,1.556177e+15,1.657324e+07,0,Benign,0
1,111395,167.49.176.28,0.0,50.165.192.168,0.0,0.0,2019-04-25 17:18:49+00:00,2033142.0,2.0,0.0,...,0.000000e+00,0.0,0.0,1.556177e+15,0.000000e+00,1.556177e+15,1.556177e+15,0,Benign,0
2,987246,230.158.52.59,0.0,177.21.192.168,0.0,0.0,2019-04-25 17:18:37+00:00,82877133.0,14.0,0.0,...,1.711593e+06,3942470.0,226402.0,1.729085e+14,5.187256e+14,1.556177e+15,6.036493e+06,0,Benign,0
3,230384,183.68.192.168,0.0,1.1.192.168,0.0,0.0,2019-04-25 17:18:42+00:00,24359.0,2.0,0.0,...,0.000000e+00,0.0,0.0,1.556177e+15,0.000000e+00,1.556177e+15,1.556177e+15,0,Benign,0
4,230209,183.41.192.168,0.0,1.1.192.168,0.0,0.0,2019-04-25 17:18:42+00:00,10239351.0,3.0,0.0,...,0.000000e+00,4053975.0,4053975.0,7.780884e+14,1.100383e+15,1.556177e+15,6.185376e+06,0,Benign,0


In [10]:
cols_to_norm = list(set(list(df.columns))  - set(list([dataset.label_col, dataset.class_num_col])) - set(dataset.drop_columns)  - set(dataset.weak_columns))

In [11]:
df[dataset.label_col].value_counts()

Label
1    2836524
0    2514059
Name: count, dtype: int64

In [12]:
if generated_ips:
    df[dataset.src_ip_col] = df[dataset.src_ip_col].apply(lambda x: socket.inet_ntoa(struct.pack('>I', random.randint(0xac100001, 0xac1f0001))))

In [13]:
if sort_timestamp:
    # df[dataset.timestamp_col] = pd.to_datetime(df[dataset.timestamp_col].str.strip(), format=dataset.timestamp_format)
    df.sort_values(dataset.timestamp_col, inplace=True)

In [14]:
if use_port_in_address:
    df[dataset.src_port_col] = df[dataset.src_port_col].astype(float).astype(int).astype(str) # to remove the decimal point
    df[dataset.src_ip_col] = df[dataset.src_ip_col] + ':' + df[dataset.src_port_col]

    df[dataset.dst_port_col] = df[dataset.dst_port_col].astype(float).astype(int).astype(str) # to remove the decimal point
    df[dataset.dst_ip_col] = df[dataset.dst_ip_col] + ':' + df[dataset.dst_port_col]

In [15]:
df.head(20)

,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,...,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label,Attack,Class
562313,179528,177.30.87.144,0.0,192.168.1.1,0.0,0.0,2019-02-04 19:53:30+00:00,115205004.0,3.0,0.0,...,0.0,0.0,0.0,5.180663e+14,8.973171e+14,1.554199e+15,1.300883e+07,0,Benign,0
562315,1006517,244.15.103.115,0.0,192.168.1.79,0.0,0.0,2019-02-04 19:54:38+00:00,6039467.0,3.0,0.0,...,0.0,0.0,0.0,1.554199e+15,0.000000e+00,1.554199e+15,1.554199e+15,0,Benign,0
562314,179528,177.30.87.144,0.0,192.168.1.1,0.0,0.0,2019-02-04 19:55:58+00:00,65536962.0,3.0,0.0,...,0.0,0.0,0.0,5.180663e+14,8.973172e+14,1.554199e+15,2.170914e+07,0,Benign,0
562317,1048404,41.238.224.122,0.0,192.168.1.190,0.0,0.0,2019-02-04 19:57:37+00:00,90177027.0,3.0,0.0,...,0.0,0.0,0.0,5.180664e+14,8.973172e+14,1.554199e+15,1.516871e+07,0,Benign,0
562316,179528,177.30.87.144,0.0,192.168.1.1,0.0,0.0,2019-02-04 19:58:04+00:00,111616761.0,3.0,0.0,...,0.0,0.0,0.0,5.180664e+14,8.973172e+14,1.554199e+15,5.447900e+07,0,Benign,0
562319,1006517,244.15.103.115,0.0,192.168.1.79,0.0,0.0,2019-02-04 19:59:07+00:00,40255700.0,2.0,0.0,...,0.0,0.0,0.0,7.770996e+14,1.098985e+15,1.554199e+15,4.025570e+07,0,Benign,0
562318,179528,177.30.87.144,0.0,192.168.1.1,0.0,0.0,2019-02-04 20:00:37+00:00,73428081.0,3.0,0.0,...,0.0,0.0,0.0,5.180664e+14,8.973173e+14,1.554199e+15,1.914878e+07,0,Benign,0
562321,179528,177.30.87.144,0.0,192.168.1.1,0.0,0.0,2019-02-04 20:03:01+00:00,56417287.0,3.0,0.0,...,0.0,0.0,0.0,5.180665e+14,8.973174e+14,1.554199e+15,2.334359e+07,0,Benign,0
562320,1006517,244.15.103.115,0.0,192.168.1.79,0.0,0.0,2019-02-04 20:03:26+00:00,98506703.0,7.0,0.0,...,0.0,1942261.0,1942261.0,2.590333e+14,6.344992e+14,1.554199e+15,6.040497e+06,0,Benign,0
563028,107367,164.91.192.168,0.0,1.152.3.122,0.0,0.0,2019-02-04 20:03:43+00:00,43687115.0,2.0,0.0,...,0.0,0.0,0.0,7.770997e+14,1.098985e+15,1.554199e+15,4.368712e+07,0,Benign,0


In [16]:
print(dataset.class_num_col)

Class


In [17]:
if multi_class:
    y = df[dataset.class_num_col]
else:
    y = df[dataset.label_col]

if sort_timestamp:
    X_tr, X_test, y_tr, y_test = train_test_split(
        df, y, test_size=test_size)
    
    X_train, X_val, y_train, y_val = train_test_split(
        X_tr, y_tr, test_size=validation_size)
else:
    X_tr, X_test, y_tr, y_test = train_test_split(
        df, y, test_size=test_size, random_state=13, stratify=y)
    
    X_train, X_val, y_train, y_val = train_test_split(
        X_tr, y_tr, test_size=validation_size, random_state=13, stratify=y_tr)

del df

In [18]:
if graph_type == "line" and use_node_features:
    add_centralities(df = X_train, new_path=None, graph_path=None, dataset=dataset, cn_measures=cn_measures, network_features=network_features, create_using=nx.MultiDiGraph())
    add_centralities(df = X_val, new_path=None, graph_path=None, dataset=dataset, cn_measures=cn_measures, network_features=network_features, create_using=nx.MultiDiGraph())
    add_centralities(df = X_test, new_path=None, graph_path=None, dataset=dataset, cn_measures=cn_measures, network_features=network_features, create_using=nx.MultiDiGraph())
    cols_to_norm = list(set(cols_to_norm) | set(network_features))
    

In [19]:
scaler = StandardScaler()

X_train[cols_to_norm] = scaler.fit_transform(X_train[cols_to_norm])
X_train['h'] = X_train[ cols_to_norm ].values.tolist()

cols_to_drop = list(set(list(X_train.columns)) - set(list([dataset.label_col, dataset.src_ip_col, dataset.dst_ip_col, dataset.class_num_col, 'h'])))
X_train.drop(cols_to_drop, axis=1, inplace=True)

X_val[cols_to_norm] = scaler.transform(X_val[cols_to_norm])
X_val['h'] = X_val[ cols_to_norm ].values.tolist()
X_val.drop(cols_to_drop, axis=1, inplace=True)

X_test[cols_to_norm] = scaler.transform(X_test[cols_to_norm])
X_test['h'] = X_test[ cols_to_norm ].values.tolist()
X_test.drop(cols_to_drop, axis=1, inplace=True)

In [20]:
if graph_type == "window" or graph_type == "line":

    create_weightless_window_graph(
        df=X_train,
        dataset=dataset,
        window_size=window_size,
        line_graph=graph_type == "line",
        folder_path=os.path.join(folder_path, "training"),
        edge_attr= ['h', dataset.label_col, dataset.class_num_col],
        file_type="pkl")
    
    create_weightless_window_graph(
        df=X_val,
        dataset=dataset,
        window_size=window_size,
        line_graph=graph_type == "line",
        folder_path=os.path.join(folder_path, "validation"),
        edge_attr= ['h', dataset.label_col, dataset.class_num_col],
        file_type="pkl")
    
    create_weightless_window_graph(
        df=X_test,
        dataset=dataset,
        window_size=window_size,
        line_graph=graph_type == "line",
        folder_path=os.path.join(folder_path, "testing"),
        edge_attr= ['h', dataset.label_col, dataset.class_num_col],
        file_type="pkl")

In [21]:
if graph_type == "flow":
	os.makedirs(folder_path, exist_ok=True)
	print(f"==>> X_train.shape: {X_train.shape}")
	print(f"==>> X_val.shape: {X_val.shape}")
	print(f"==>> X_test.shape: {X_test.shape}")


==>> X_train.shape: (4333971, 5)
==>> X_val.shape: (481553, 5)
==>> X_test.shape: (535059, 5)


In [22]:
if graph_type == "flow":
    graph_name = "training_graph"

    G = nx.from_pandas_edgelist(X_train, dataset.src_ip_col, dataset.dst_ip_col, ['h',dataset.label_col, dataset.class_num_col], create_using=nx.MultiDiGraph())
    
    if use_node_features:
        add_centralities_as_node_features(df=None, G=G, graph_path=None, dataset=dataset, cn_measures=cn_measures)
        
        for node in G.nodes():
            centralities = []
            for centrality in cn_measures:
                centralities.append(G.nodes[node].get(centrality, 0)) # Default to 0 if missing
                
                # Combine features into a single vector
            n_feats = np.array(centralities, dtype=np.float32)
            
            # Add the new feature to the node
            G.nodes[node]["n_feats"] = n_feats
            
    # get netowrk properties
    graph_measures = calculate_graph_measures(G, f"{folder_path}/{graph_name}_measures.json", verbose=True)
    print(f"==>> graph_measures: {graph_measures}")

    # graph_measures = calculate_graph_measures(nx.DiGraph(G), "datasets/" + name + "/training_graph_simple_measures.json", verbose=True)
    # print(f"==>> graph_measures: {graph_measures}")

    with open(f"{folder_path}/{graph_name}.pkl", "wb") as f:
        pickle.dump(G, f)

calculated betweenness
calculated degree
calculated closeness
calculated pagerank
calculated k_truss
==>> calculated degrees, in 0.2526018190001196 seconds
==>> graph_measures: {'number_of_nodes': 132311, 'number_of_edges': 4333971, 'max_degree': 1674190, 'avg_degree': 65.51187731934608, 'density': 0.0002475696369108385}


In [23]:
if graph_type == "flow":
    graph_name = "validation_graph"

    G = nx.from_pandas_edgelist(X_val, dataset.src_ip_col, dataset.dst_ip_col, ['h',dataset.label_col, dataset.class_num_col], create_using=nx.MultiDiGraph())
    
    if use_node_features:
        add_centralities_as_node_features(df=None, G=G, graph_path=None, dataset=dataset, cn_measures=cn_measures)
        
        for node in G.nodes():
            centralities = []
            for centrality in cn_measures:
                centralities.append(G.nodes[node].get(centrality, 0)) # Default to 0 if missing
                
                # Combine features into a single vector
            n_feats = np.array(centralities, dtype=np.float32)
            
            # Add the new feature to the node
            G.nodes[node]["n_feats"] = n_feats
            
    # get netowrk properties
    graph_measures = calculate_graph_measures(G, f"{folder_path}/{graph_name}_measures.json", verbose=True)
    print(f"==>> graph_measures: {graph_measures}")

    # graph_measures = calculate_graph_measures(nx.DiGraph(G), "datasets/" + name + "/training_graph_simple_measures.json", verbose=True)
    # print(f"==>> graph_measures: {graph_measures}")

    with open(f"{folder_path}/{graph_name}.pkl", "wb") as f:
        pickle.dump(G, f)

calculated betweenness
calculated degree
calculated closeness
calculated pagerank
calculated k_truss
==>> calculated degrees, in 0.04900716699921759 seconds
==>> graph_measures: {'number_of_nodes': 37769, 'number_of_edges': 481553, 'max_degree': 186329, 'avg_degree': 25.499907331409357, 'density': 0.0003375861487424454}


In [24]:
if graph_type == "flow":
    graph_name = "testing_graph"
    
    G = nx.from_pandas_edgelist(X_test, dataset.src_ip_col, dataset.dst_ip_col, ['h', dataset.label_col, dataset.class_num_col], create_using=nx.MultiDiGraph())
    
    if use_node_features:
        add_centralities_as_node_features(df=None, G=G, graph_path=None, dataset=dataset, cn_measures=cn_measures)
        
        for node in G.nodes():
            centralities = []
            for centrality in cn_measures:
                centralities.append(G.nodes[node].get(centrality, 0)) # Default to 0 if missing
                
                # Combine features into a single vector
            n_feats = np.array(centralities, dtype=np.float32)
            
            # Add the new feature to the node
            G.nodes[node]["n_feats"] = n_feats
            
    graph_measures = calculate_graph_measures(G, f"{folder_path}/{graph_name}_measures.json", verbose=True)
    print(f"==>> graph_measures: {graph_measures}")
    
    # graph_measures = calculate_graph_measures(nx.DiGraph(G_test), "datasets/" + name + "/testing_graph_simple_measures.json", verbose=True)
    # print(f"==>> graph_measures: {graph_measures}")
    
    with open(f"{folder_path}/{graph_name}.pkl", "wb") as f:
        pickle.dump(G, f)

calculated betweenness
calculated degree
calculated closeness
calculated pagerank
calculated k_truss
==>> calculated degrees, in 0.059191026000007696 seconds
==>> graph_measures: {'number_of_nodes': 41240, 'number_of_edges': 535059, 'max_degree': 206269, 'avg_degree': 25.94854510184287, 'density': 0.00031461171587384356}


In [25]:
import json

print("-------------------------- SAVING VARIABLES -------------------------- ")
data = {
    "dataset_path": dataset_path,
    "folder_path": folder_path,
    "name": name,
    "g_type": g_type
}

# Save to JSON
with open("config.json", "w") as f:
    json.dump(data, f, indent=4)


-------------------------- SAVING VARIABLES -------------------------- 


In [26]:
print(G)
print(y)
print(X_train, X_val, y_train, y_val)
del G
del y 
del X_train, X_val, y_train, y_val

MultiDiGraph with 41240 nodes and 535059 edges
562313    0
562315    0
562314    0
562317    0
562316    0
         ..
291672    0
329941    0
329974    0
341650    0
334132    0
Name: Class, Length: 5350583, dtype: int64
                Src IP         Dst IP  Label  Class  \
2119601   192.168.1.39  192.168.1.190      1      9   
179170   26.99.192.168   1.169.74.125      0      0   
653771   192.168.1.193   192.168.1.33      0      0   
4655754  192.168.1.195   192.168.1.39      0      0   
1725979   192.168.1.36  192.168.1.152      1      9   
...                ...            ...    ...    ...   
1398075   192.168.1.36  52.28.231.150      0      0   
436919   192.168.1.193   192.168.1.33      1      1   
4973201   192.168.1.35  192.168.1.190      1      6   
2925346   192.168.1.35  192.168.1.195      1      6   
903134    192.168.1.32  192.168.1.195      1      6   

                                                         h  
2119601  [0.0015098238038789129, -0.031140547671802212,.

In [27]:
print(scaler)
del scaler

StandardScaler()


In [28]:
import psutil

ram_gb = psutil.virtual_memory().used / 1e9
print(f"Current RAM usage: {ram_gb:.2f} GB")


Current RAM usage: 17.04 GB


In [29]:
import gc
gc.collect()


625960

In [30]:
print("-------------------------- CLEARING THE MEMORY -------------------------- ")
print("Clear all variables (like restarting kernel)")
%reset -f


-------------------------- CLEARING THE MEMORY -------------------------- 
Clear all variables (like restarting kernel)


In [31]:
import psutil
ram_gb = psutil.virtual_memory().used / 1e9
print(f"Current RAM usage: {ram_gb:.2f} GB")


Current RAM usage: 5.35 GB


<h1>TRAINNING</h1>

In [32]:
print("-------------------------- RETRIVING VARIABLES --------------------------")
import json
with open("config.json", "r") as f:
    config = json.load(f)

# Access the values
dataset_path = config["dataset_path"]
folder_path = config["folder_path"]
name = config["name"]
g_type = config["g_type"]


-------------------------- RETRIVING VARIABLES --------------------------


In [33]:
print(dataset_path)
print(folder_path)
print(name)
print(g_type)

/kaggle/input/cic-ton-iot-gnn
/kaggle/working/cic_ton_iot/flow__multi_class__n_feats__sorted
cic_ton_iot
flow__multi_class__n_feats__sorted


In [34]:
dataset_name = name
dataset_folder = dataset_path
graph_folder=folder_path
model_path = "/kaggle/input/gnn-nids-wandb/pytorch/default/6/"

using_wandb = False
# save_top_k = 5
save_top_k = 1
# save_top_k = 4 save best 4 test model from the traininng based on f1_score 
# early_stopping_patience = max_epochs = 20
early_stopping_patience = max_epochs = 500
# early_stopping_patience = 200 all - epochs = 500 - lr = 0.005
learning_rate = 0.005
weight_decay = 0.01
ndim_out = [32, 32]
num_layers = 2
number_neighbors = [25, 10]
dropout = 0.5
residual = True
multi_class = True
aggregation = "mean"
# aggregation = "sum"

use_centralities_nfeats = False

# g_type = "flow"

# if multi_class:
#     g_type += "__multi_class"

# if use_centralities_nfeats:
#     g_type += "__n_feats"

# g_type += "__unsorted"

In [35]:
pip uninstall -y torch torchvision torchaudio torchdata pytorch-lightning --quiet

Note: you may need to restart the kernel to use updated packages.


In [36]:
pip install torch==2.1.0+cu118 torchvision==0.16.0+cu118 torchaudio==2.1.0+cu118 -f https://download.pytorch.org/whl/torch_stable.html --quiet

Note: you may need to restart the kernel to use updated packages.


In [37]:
import torch
print("torch version:", torch.__version__)
print("CUDA available?", torch.cuda.is_available())

torch version: 2.1.0+cu118
CUDA available? True


In [38]:
pip install dgl -f https://data.dgl.ai/wheels/torch-2.1/cu118/repo.html --quiet

Note: you may need to restart the kernel to use updated packages.


In [39]:
!apt update -qq 2>/dev/null >/dev/null;
!apt install -y libcusparse11 -qq 2>/dev/null >/dev/null;

In [40]:
pip install --upgrade -q pytorch-lightning==2.5.0

Note: you may need to restart the kernel to use updated packages.


In [41]:
!pip install --upgrade -q wandb==0.19.6

In [42]:
# import wandb
# wandb.login(key="c20e6d52d61e51ff11f4391b5870097badb9092f")
import wandb
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("mohammad_wandb_secret")

wandb.login(key=secret_value_0)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: mohammad-fleity (mohammad-fleity-lebanese-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [43]:
import os
import json
import pickle
import time
import timeit
import importlib.util
import sys
import random
import numpy as np
import csv
os.environ["DGLBACKEND"] = "pytorch"

import torch.nn as nn
import torch
import warnings
import wandb
import torch.nn.functional as F
import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping

seed = 42  # or any constant value
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

run_dtime = time.strftime("%Y%m%d-%H%M%S")

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/gnn-nids/pytorch/default/1/time_efficiency.ipynb
/kaggle/input/gnn-nids/pytorch/default/1/.gitignore
/kaggle/input/gnn-nids/pytorch/default/1/prepare_graph_files.ipynb
/kaggle/input/gnn-nids/pytorch/default/1/main.py
/kaggle/input/gnn-nids/pytorch/default/1/README.md
/kaggle/input/gnn-nids/pytorch/default/1/dataset_properties.ipynb
/kaggle/input/gnn-nids/pytorch/default/1/pre_processing.ipynb
/kaggle/input/gnn-nids/pytorch/default/1/concat_properties.py
/kaggle/input/gnn-nids/pytorch/default/1/requirements.txt
/kaggle/input/gnn-nids/pytorch/default/1/graph_properties.ipynb
/kaggle/input/gnn-nids/pytorch/default/1/results_analysis.ipynb
/kaggle/input/gnn-nids/pytorch/default/1/testing_dfs/cic_ton_iot_5_percent.parquet
/kaggle/input/gnn-nids/pytorch/default/1/testing_dfs/cic_ids_2017_5_percent.parquet
/kaggle/input/gnn-nids/pytorch/default/1/src/lightning_model.py
/kaggle/input/gnn-nids/pytorch/default/1/src/utils.py
/kaggle/input/gnn-nids/pytorch/default/1/src/models.py
/k

In [44]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [45]:
# import kagglehub

# # Download latest version
# path = kagglehub.model_download("mortadaphdtermos/gnn-nids-wandb/pyTorch/default")

# print("Path to model files:", path)

In [46]:
def add_lib(module_name, path):
    spec = importlib.util.spec_from_file_location(module_name, path)
    dataset_info = importlib.util.module_from_spec(spec)
    sys.modules[module_name] = dataset_info
    spec.loader.exec_module(dataset_info)

add_lib("dataset_info", model_path + "dataset_info.py")
add_lib("lightning_data", model_path + "lightning_data.py")
add_lib("lightning_model", model_path + "lightning_model.py")
add_lib("models", model_path + "models.py")

In [47]:
from models import EGAT, EGCN, EGRAPHSAGE
from lightning_model import GraphModel
from lightning_data import GraphDataModule
from dataset_info import datasets

In [48]:
print(os.path.join(dataset_folder, "labels_names.pkl"))

/kaggle/input/cic-ton-iot-gnn/labels_names.pkl


<h2>E_GAT</h2>

In [49]:
# print("----------------------- TRAINING E_GAT --------------------------")

In [ ]:
warnings.filterwarnings("ignore", ".*does not have many workers.*")

dataset = datasets[dataset_name]

# Hyperparameters

activation = F.relu
graphs_folder=graph_folder
# graphs_folder = os.path.join(dataset_folder, g_type)
# if dataset_name == "cic_ids_2017":
#     graphs_folder = "/kaggle/working/cic_ids_2017/flow__multi_class__n_feats__unsorted"
logs_folder = os.path.join("logs", dataset.name)
os.makedirs(logs_folder, exist_ok=True)
wandb_runs_path = os.path.join("logs", "wandb_runs")
os.makedirs(wandb_runs_path, exist_ok=True)

labels_mapping = {0: "Normal", 1: "Attack"}
num_classes = 2
if multi_class:
    with open(os.path.join(dataset_folder, "labels_names.pkl"), "rb") as f:
    # /kaggle/input/cic-ids-2017-gnn-2/labels_names.pkl
    # with open("/kaggle/input/cic-ids-2017-gnn-2/labels_names.pkl", "rb") as f:
        labels_names = pickle.load(f)
    labels_mapping = labels_names[0]
num_classes = len(labels_mapping)

dataset_kwargs = dict(
    use_node_features=use_centralities_nfeats,
    multi_class=True,
    using_masking=False,
    masked_class=2,
    num_workers=0,
    label_col=dataset.label_col,
    class_num_col=dataset.class_num_col,
    device='cuda' if torch.cuda.is_available() else "cpu"
)

data_module = GraphDataModule(
    graphs_folder, batch_size=1, **dataset_kwargs)
data_module.setup()

ndim = next(iter(data_module.train_dataloader())).ndata["h"].shape[-1]
edim = next(iter(data_module.train_dataloader())).edata['h'].shape[-1]

# my_models = {
#     # "e_gcn": EGCN(ndim, edim, ndim_out, num_layers, activation,
#     #               dropout, residual, num_classes),
#     # f"e_graphsage_{aggregation}": EGRAPHSAGE(ndim, edim, ndim_out, num_layers, activation, dropout,
#     #                                          residual, num_classes, num_neighbors=number_neighbors, aggregation=aggregation),
#     # f"e_graphsage_{aggregation}_no_sampling": EGRAPHSAGE(ndim, edim, ndim_out, num_layers, activation, dropout,
#     #                                                      residual, num_classes, num_neighbors=None, aggregation=aggregation),
#     # "e_gat_sampling": EGAT(ndim, edim, ndim_out, num_layers, activation, dropout,
#                   # residual, num_classes, num_neighbors=number_neighbors),
#     "e_gat_no_sampling": EGAT(ndim, edim, ndim_out, num_layers, activation, dropout, residual, num_classes, num_neighbors=None),
# }
my_models={
    "e_gcn": EGCN(ndim, edim, ndim_out, num_layers, activation, dropout, residual, num_classes),
    f"e_graphsage_{aggregation}_no_sampling": EGRAPHSAGE(ndim, edim, ndim_out, num_layers, activation, dropout,residual, num_classes, num_neighbors=None, aggregation=aggregation),
    f"e_graphsage_{aggregation}": EGRAPHSAGE(
        ndim, edim, ndim_out, num_layers, activation, dropout,
        residual, num_classes, num_neighbors=number_neighbors,
        aggregation=aggregation
    ),
    f"e_graphsage_sum_no_sampling": EGRAPHSAGE(ndim, edim, ndim_out, num_layers, activation, dropout,residual, num_classes, num_neighbors=None, aggregation="sum"),
    f"e_graphsage_sum": EGRAPHSAGE(
        ndim, edim, ndim_out, num_layers, activation, dropout,
        residual, num_classes, num_neighbors=number_neighbors,
        aggregation="sum"
    ),
    "e_gat_sampling": EGAT(
        ndim, edim, ndim_out, num_layers, activation, dropout,
        residual, num_classes, num_neighbors=number_neighbors
    ),
    "e_gat_no_sampling": EGAT(
        ndim, edim, ndim_out, num_layers, activation, dropout,
        residual, num_classes, num_neighbors=None
    ),
}

criterion = nn.CrossEntropyLoss(data_module.train_dataset.class_weights)

for model_name, model in my_models.items():
    print("----------------------- TRAINING ",model_name," --------------------------")
    config = {
        "run_dtime": run_dtime,
        "type": "GNN",
        "model_name": model_name,
        "max_epochs": max_epochs,
        "learning_rate": learning_rate,
        "weight_decay": weight_decay,
        "ndim_out": ndim_out,
        "num_layers": num_layers,
        "number_neighbors": number_neighbors,
        "activation": activation.__name__,
        "dropout": dropout,
        "residual": residual,
        "multi_class": multi_class,
        "aggregation": aggregation,
        "early_stopping_patience": early_stopping_patience,
        "use_centralities_nfeats": use_centralities_nfeats,}

    process = psutil.Process(os.getpid())  # current process
    start_time = time.time()
    start_mem = process.memory_info().rss / (1024 ** 2)  # in MB
    graph_model = GraphModel(model, criterion, learning_rate, config, model_name,
                             labels_mapping, weight_decay=weight_decay, using_wandb=using_wandb, norm=False, multi_class=True)

    
    if using_wandb:
        wandb_logger = WandbLogger(
            project=f"GNN-Analysis-{dataset.name}", # Project Name Wandb
            name=f"{model_name}---{g_type}",
            config=config,
            save_dir=wandb_runs_path
        )
    else:
        wandb_logger = None
        
    f1_checkpoint_callback = ModelCheckpoint(
        monitor="val_f1_score",
        mode="max",
        filename="best-val-f1-{epoch:02d}-{val_f1_score:.2f}",
        save_top_k=save_top_k,
        verbose=False,
    )
    early_stopping_callback = EarlyStopping(
        monitor="val_loss",
        mode="min",
        patience=early_stopping_patience,
        verbose=False,
    )


    trainer = pl.Trainer(
        max_epochs=max_epochs,
        num_sanity_val_steps=0,
        log_every_n_steps=0,
        callbacks=[
            f1_checkpoint_callback,
            early_stopping_callback
        ],
        default_root_dir=logs_folder,
        logger=wandb_logger if using_wandb else None,
    )

    trainer.fit(graph_model, datamodule=data_module)
    
    test_results = []
    
    # for i, k in enumerate(f1_checkpoint_callback.best_k_models.keys()):
    #     graph_model.test_prefix = f"best_f1_{i}"
    #     results = trainer.test(
    #         graph_model, datamodule=data_module, ckpt_path=k)
    #     test_results.append(results[0][f"best_f1_{i}_test_f1"])

    # logs = {
    #     "median_f1_of_best_f1": np.median(test_results),
    #     "max_f1_of_best_f1": np.max(test_results),
    #     "avg_f1_of_best_f1": np.mean(test_results)
    # }

    # if using_wandb:
    #     wandb.log(logs)
    #     wandb.finish()

    print(process)
    for i, k in enumerate(f1_checkpoint_callback.best_k_models.keys()):
        graph_model.test_prefix = f"best_f1_{i}"
        results = trainer.test(graph_model, datamodule=data_module, ckpt_path=k)
        test_results.append(results[0][f"best_f1_{i}_test_f1"])

        print(f"\n--- Test Run {i+1} ---")
        print("graph_model: ", graph_model)
        print("data_module: ", data_module)
        print("results: ", results)
        logs = {
        "median_f1_of_best_f1": np.median(test_results),
        "max_f1_of_best_f1": np.max(test_results),
        "avg_f1_of_best_f1": np.mean(test_results),
            
    }
    print("test_results: ", test_results)
    end_time = time.time()
    end_mem = process.memory_info().rss / (1024 ** 2)  # in MB
    
    elapsed_time = end_time - start_time
    mem_used = end_mem - start_mem
    print(f"Elapsed time: {elapsed_time:.2f} seconds")
    print(f"Memory used: {mem_used:.2f} MB (current RSS: {end_mem:.2f} MB)")
    metric_dir= "/kaggle/working/temp"
    os.makedirs(metric_dir, exist_ok=True)
    
    csv_path = os.path.join(metric_dir, f"{model_name}_metrics.csv")
    
    with open(csv_path, mode="w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["TEST_ACCURACY", test_results[0]])
        writer.writerow(["MEMORY_USED", f"{mem_used:.2f} MB"])
        writer.writerow(["TIME_CONSUMED", f"{elapsed_time:.2f} s."])
    
    print(f"Metrics saved to {csv_path}")


    if using_wandb:
        wandb.log(logs)
        wandb.finish()
    else:
        trainer.logger.log_metrics(logs, step=trainer.global_step)

In [ ]:
import os
import json
import pandas as pd

# labels_map = {
#     "0": "BENIGN",
#     "1": "Bot",
#     "2": "DDoS",
#     "3": "DoS GoldenEye",
#     "4": "DoS Hulk",
#     "5": "DoS Slowhttptest",
#     "6": "DoS slowloris",
#     "7": "FTP-Patator",
#     "8": "Heartbleed",
#     "9": "Infiltration",
#     "10": "PortScan",
#     "11": "SSH-Patator",
#     "12": "Web Attack \ufffd Brute Force",
#     "13": "Web Attack \ufffd Sql Injection",
#     "14": "Web Attack \ufffd XSS"
# }
labels_map = {
    "0": "Benign",
    "1": "backdoor",
    "2": "ddos",
    "3": "dos",
    "4": "injection",
    "5": "mitm",
    "6": "password",
    "7": "ransomware",
    "8": "scanning",
    "9": "xss"
}
def json_to_csv_per_file(json_folder, output_folder):
    os.makedirs(output_folder, exist_ok=True)

    for filename in os.listdir(json_folder):
        if filename.endswith(".json"):
            filepath = os.path.join(json_folder, filename)
            with open(filepath, "r") as f:
                data = json.load(f)

            rows = []
            # --- Per-class metrics ---
            classification_report=data["classification_report"]
            # print(classification_report)
            if "results_fpr_fnr" in data and "per_class" in data["results_fpr_fnr"]:
                # print(data["results_fpr_fnr"])
                for cls, metrics in data["results_fpr_fnr"]["per_class"].items():
                    attack=labels_map.get(cls,cls)
                    print(f"{attack}")
                    metrics_d=classification_report.get(attack)
                    row = {
                        "ATTACK TYPE": attack,
                        "precision": metrics_d.get("precision"),
                        "recall": metrics_d.get("recall"),
                        "f1_score": metrics_d.get("f1-score"),
                        "support": metrics_d.get("support"),
                        "FPR": metrics.get("FPR"),
                        "FNR": metrics.get("FNR"),
                    }
                    # print()
                    # print(metrics_d.get("precision"))
                    # print()
                    rows.append(row)

            df_classes = pd.DataFrame(rows)
            weighted_avg=classification_report['weighted avg']
            # print(data)
            # --- Global / test metrics ---
            global_metrics = {
                "test_weighted_f1": data.get("test_weighted_f1"),
                'test_accuracy':data.get('accuracy'),
                'precision': weighted_avg['precision'],
                'recall': weighted_avg['recall'], 
                'support': weighted_avg['support'],
                # "test_fpr": data.get("test_fpr"),
                # "test_fnr": data.get("test_fnr"),
                "global_FPR": data["results_fpr_fnr"]["global"].get("FPR") if "results_fpr_fnr" in data and "global" in data["results_fpr_fnr"] else None,
                "global_FNR": data["results_fpr_fnr"]["global"].get("FNR") if "results_fpr_fnr" in data and "global" in data["results_fpr_fnr"] else None,
                "time_consumption":None,
                "memory_consumption":None,
            }
            print(global_metrics)
            df_global = pd.DataFrame([global_metrics])

            # --- Save CSV ---
            csv_output = os.path.join(output_folder, filename.replace(".json", ".csv"))
            with open(csv_output, "w") as f:
                f.write("### Per-class Metrics\n")
            df_classes.to_csv(csv_output, mode="a", index=False)
            with open(csv_output, "a") as f:
                f.write("\n### Global Metrics\n")
            df_global.to_csv(csv_output, mode="a", index=False)

            print(f"Saved {csv_output}")
json_folder = "/kaggle/working/temp/"
output_folder = "/kaggle/working/temp/csv_results"
json_to_csv_per_file(json_folder, output_folder)


<h2>E_GAT Sampling</h2>

In [ ]:
# print("----------------------- TRAINING E_GAT Sampling--------------------------")


In [ ]:
# warnings.filterwarnings("ignore", ".*does not have many workers.*")

# dataset = datasets[dataset_name]

# # Hyperparameters

# activation = F.relu
# graphs_folder = graph_folder
# if dataset_name == "cic_ids_2017":
#     graphs_folder = "/kaggle/working/cic_ids_2017/flow__multi_class__n_feats__unsorted"
# logs_folder = os.path.join("logs", dataset.name)
# os.makedirs(logs_folder, exist_ok=True)
# wandb_runs_path = os.path.join("logs", "wandb_runs")
# os.makedirs(wandb_runs_path, exist_ok=True)

# labels_mapping = {0: "Normal", 1: "Attack"}
# num_classes = 2
# if multi_class:
#     with open(os.path.join(dataset_folder, "labels_names.pkl"), "rb") as f:
#     # /kaggle/input/cic-ids-2017-gnn-2/labels_names.pkl
#     # with open("/kaggle/input/cic-ids-2017-gnn-2/labels_names.pkl", "rb") as f:
#         labels_names = pickle.load(f)
#     labels_mapping = labels_names[0]
# num_classes = len(labels_mapping)

# dataset_kwargs = dict(
#     use_node_features=use_centralities_nfeats,
#     multi_class=True,
#     using_masking=False,
#     masked_class=2,
#     num_workers=0,
#     label_col=dataset.label_col,
#     class_num_col=dataset.class_num_col,
#     device='cuda' if torch.cuda.is_available() else "cpu"
# )

# data_module = GraphDataModule(
#     graphs_folder, batch_size=1, **dataset_kwargs)
# data_module.setup()

# ndim = next(iter(data_module.train_dataloader())).ndata["h"].shape[-1]
# edim = next(iter(data_module.train_dataloader())).edata['h'].shape[-1]

# my_models = {
#     # "e_gcn": EGCN(ndim, edim, ndim_out, num_layers, activation,
#     #               dropout, residual, num_classes),
#     # f"e_graphsage_{aggregation}": EGRAPHSAGE(ndim, edim, ndim_out, num_layers, activation, dropout,
#     #                                          residual, num_classes, num_neighbors=number_neighbors, aggregation=aggregation),
#     # f"e_graphsage_{aggregation}_no_sampling": EGRAPHSAGE(ndim, edim, ndim_out, num_layers, activation, dropout,
#     #                                                      residual, num_classes, num_neighbors=None, aggregation=aggregation),
#     "e_gat_sampling": EGAT(ndim, edim, ndim_out, num_layers, activation, dropout,
#                   residual, num_classes, num_neighbors=number_neighbors),
#     # "e_gat_no_sampling": EGAT(ndim, edim, ndim_out, num_layers, activation, dropout, residual, num_classes, num_neighbors=None),
# }

# criterion = nn.CrossEntropyLoss(data_module.train_dataset.class_weights)

# for model_name, model in my_models.items():
        
#     config = {
#         "run_dtime": run_dtime,
#         "type": "GNN",
#         "model_name": model_name,
#         "max_epochs": max_epochs,
#         "learning_rate": learning_rate,
#         "weight_decay": weight_decay,
#         "ndim_out": ndim_out,
#         "num_layers": num_layers,
#         "number_neighbors": number_neighbors,
#         "activation": activation.__name__,
#         "dropout": dropout,
#         "residual": residual,
#         "multi_class": multi_class,
#         "aggregation": aggregation,
#         "early_stopping_patience": early_stopping_patience,
#         "use_centralities_nfeats": use_centralities_nfeats,}

#     graph_model = GraphModel(model, criterion, learning_rate, config, model_name,
#                              labels_mapping, weight_decay=weight_decay, using_wandb=using_wandb, norm=False, multi_class=True)

    
#     if using_wandb:
#         wandb_logger = WandbLogger(
#             project=f"GNN-Analysis-{dataset.name}", # Project Name Wandb
#             name=f"{model_name}---{g_type}",
#             config=config,
#             save_dir=wandb_runs_path
#         )
#     else:
#         wandb_logger = None
        
#     f1_checkpoint_callback = ModelCheckpoint(
#         monitor="val_f1_score",
#         mode="max",
#         filename="best-val-f1-{epoch:02d}-{val_f1_score:.2f}",
#         save_top_k=save_top_k,
#         verbose=False,
#     )
#     early_stopping_callback = EarlyStopping(
#         monitor="val_loss",
#         mode="min",
#         patience=early_stopping_patience,
#         verbose=False,
#     )

#     trainer = pl.Trainer(
#         max_epochs=max_epochs,
#         num_sanity_val_steps=0,
#         log_every_n_steps=0,
#         callbacks=[
#             f1_checkpoint_callback,
#             early_stopping_callback
#         ],
#         default_root_dir=logs_folder,
#         logger=wandb_logger if using_wandb else None,
#     )

#     trainer.fit(graph_model, datamodule=data_module)
    
#     test_results = []
    
#     for i, k in enumerate(f1_checkpoint_callback.best_k_models.keys()):
#         graph_model.test_prefix = f"best_f1_{i}"
#         results = trainer.test(
#             graph_model, datamodule=data_module, ckpt_path=k)
#         test_results.append(results[0][f"best_f1_{i}_test_f1"])

#     logs = {
#         "median_f1_of_best_f1": np.median(test_results),
#         "max_f1_of_best_f1": np.max(test_results),
#         "avg_f1_of_best_f1": np.mean(test_results)
#     }

#     if using_wandb:
#         wandb.log(logs)
#         wandb.finish()
#     else:
#         trainer.logger.log_metrics(logs, step=trainer.global_step)

<h2>EGraphSAGE mean</h2>

In [ ]:
# print("----------------------- TRAINING EGraphSAGE mean --------------------------")


In [ ]:
# warnings.filterwarnings("ignore", ".*does not have many workers.*")

# dataset = datasets[dataset_name]

# # Hyperparameters

# activation = F.relu
# graphs_folder = graph_folder
# if dataset_name == "cic_ids_2017":
#     graphs_folder = "/kaggle/working/cic_ids_2017/flow__multi_class__n_feats__unsorted"
# logs_folder = os.path.join("logs", dataset.name)
# os.makedirs(logs_folder, exist_ok=True)
# wandb_runs_path = os.path.join("logs", "wandb_runs")
# os.makedirs(wandb_runs_path, exist_ok=True)

# labels_mapping = {0: "Normal", 1: "Attack"}
# num_classes = 2
# if multi_class:
#     with open(os.path.join(dataset_folder, "labels_names.pkl"), "rb") as f:
#     # /kaggle/input/cic-ids-2017-gnn-2/labels_names.pkl
#     # with open("/kaggle/input/cic-ids-2017-gnn-2/labels_names.pkl", "rb") as f:
#         labels_names = pickle.load(f)
#     labels_mapping = labels_names[0]
# num_classes = len(labels_mapping)

# dataset_kwargs = dict(
#     use_node_features=use_centralities_nfeats,
#     multi_class=True,
#     using_masking=False,
#     masked_class=2,
#     num_workers=0,
#     label_col=dataset.label_col,
#     class_num_col=dataset.class_num_col,
#     device='cuda' if torch.cuda.is_available() else "cpu"
# )

# data_module = GraphDataModule(
#     graphs_folder, batch_size=1, **dataset_kwargs)
# data_module.setup()

# ndim = next(iter(data_module.train_dataloader())).ndata["h"].shape[-1]
# edim = next(iter(data_module.train_dataloader())).edata['h'].shape[-1]

# my_models = {
#     # "e_gcn": EGCN(ndim, edim, ndim_out, num_layers, activation,
#     #               dropout, residual, num_classes),
#     # f"e_graphsage_{aggregation}": EGRAPHSAGE(ndim, edim, ndim_out, num_layers, activation, dropout,
#     #                                          residual, num_classes, num_neighbors=number_neighbors, aggregation=aggregation),
#     f"e_graphsage_{aggregation}_no_sampling": EGRAPHSAGE(ndim, edim, ndim_out, num_layers, activation, dropout,
#                                                          residual, num_classes, num_neighbors=None, aggregation=aggregation),
#     # "e_gat_sampling": EGAT(ndim, edim, ndim_out, num_layers, activation, dropout,
#     #               residual, num_classes, num_neighbors=number_neighbors),
#     # "e_gat_no_sampling": EGAT(ndim, edim, ndim_out, num_layers, activation, dropout, residual, num_classes, num_neighbors=None),
# }

# criterion = nn.CrossEntropyLoss(data_module.train_dataset.class_weights)

# for model_name, model in my_models.items():
        
#     config = {
#         "run_dtime": run_dtime,
#         "type": "GNN",
#         "model_name": model_name,
#         "max_epochs": max_epochs,
#         "learning_rate": learning_rate,
#         "weight_decay": weight_decay,
#         "ndim_out": ndim_out,
#         "num_layers": num_layers,
#         "number_neighbors": number_neighbors,
#         "activation": activation.__name__,
#         "dropout": dropout,
#         "residual": residual,
#         "multi_class": multi_class,
#         "aggregation": aggregation,
#         "early_stopping_patience": early_stopping_patience,
#         "use_centralities_nfeats": use_centralities_nfeats,}

#     graph_model = GraphModel(model, criterion, learning_rate, config, model_name,
#                              labels_mapping, weight_decay=weight_decay, using_wandb=using_wandb, norm=False, multi_class=True)

    
#     if using_wandb:
#         wandb_logger = WandbLogger(
#             project=f"GNN-Analysis-{dataset.name}", # Project Name Wandb
#             name=f"{model_name}---{g_type}",
#             config=config,
#             save_dir=wandb_runs_path
#         )
#     else:
#         wandb_logger = None
        
#     f1_checkpoint_callback = ModelCheckpoint(
#         monitor="val_f1_score",
#         mode="max",
#         filename="best-val-f1-{epoch:02d}-{val_f1_score:.2f}",
#         save_top_k=save_top_k,
#         verbose=False,
#     )
#     early_stopping_callback = EarlyStopping(
#         monitor="val_loss",
#         mode="min",
#         patience=early_stopping_patience,
#         verbose=False,
#     )

#     trainer = pl.Trainer(
#         max_epochs=max_epochs,
#         num_sanity_val_steps=0,
#         log_every_n_steps=0,
#         callbacks=[
#             f1_checkpoint_callback,
#             early_stopping_callback
#         ],
#         default_root_dir=logs_folder,
#         logger=wandb_logger if using_wandb else None,
#     )

#     trainer.fit(graph_model, datamodule=data_module)
    
#     test_results = []
    
#     for i, k in enumerate(f1_checkpoint_callback.best_k_models.keys()):
#         graph_model.test_prefix = f"best_f1_{i}"
#         results = trainer.test(
#             graph_model, datamodule=data_module, ckpt_path=k)
#         test_results.append(results[0][f"best_f1_{i}_test_f1"])

#     logs = {
#         "median_f1_of_best_f1": np.median(test_results),
#         "max_f1_of_best_f1": np.max(test_results),
#         "avg_f1_of_best_f1": np.mean(test_results)
#     }

#     if using_wandb:
#         wandb.log(logs)
#         wandb.finish()
#     else:
#         trainer.logger.log_metrics(logs, step=trainer.global_step)

<h2>EGraphSAGE sum</h2>

In [ ]:
# print("----------------------- TRAINING EGraphSAGE sum --------------------------")

In [ ]:
# warnings.filterwarnings("ignore", ".*does not have many workers.*")

# dataset = datasets[dataset_name]

# # Hyperparameters
# aggregation="sum"

# activation = F.relu
# graphs_folder = graph_folder
# if dataset_name == "cic_ids_2017":
#     graphs_folder = "/kaggle/working/cic_ids_2017/flow__multi_class__n_feats__unsorted"
# logs_folder = os.path.join("logs", dataset.name)
# os.makedirs(logs_folder, exist_ok=True)
# wandb_runs_path = os.path.join("logs", "wandb_runs")
# os.makedirs(wandb_runs_path, exist_ok=True)

# labels_mapping = {0: "Normal", 1: "Attack"}
# num_classes = 2
# if multi_class:
#     with open(os.path.join(dataset_folder, "labels_names.pkl"), "rb") as f:
#     # /kaggle/input/cic-ids-2017-gnn-2/labels_names.pkl
#     # with open("/kaggle/input/cic-ids-2017-gnn-2/labels_names.pkl", "rb") as f:
#         labels_names = pickle.load(f)
#     labels_mapping = labels_names[0]
# num_classes = len(labels_mapping)

# dataset_kwargs = dict(
#     use_node_features=use_centralities_nfeats,
#     multi_class=True,
#     using_masking=False,
#     masked_class=2,
#     num_workers=0,
#     label_col=dataset.label_col,
#     class_num_col=dataset.class_num_col,
#     device='cuda' if torch.cuda.is_available() else "cpu"
# )

# data_module = GraphDataModule(
#     graphs_folder, batch_size=1, **dataset_kwargs)
# data_module.setup()

# ndim = next(iter(data_module.train_dataloader())).ndata["h"].shape[-1]
# edim = next(iter(data_module.train_dataloader())).edata['h'].shape[-1]

# my_models = {
#     # "e_gcn": EGCN(ndim, edim, ndim_out, num_layers, activation,
#     #               dropout, residual, num_classes),
#     # f"e_graphsage_{aggregation}": EGRAPHSAGE(ndim, edim, ndim_out, num_layers, activation, dropout,
#     #                                          residual, num_classes, num_neighbors=number_neighbors, aggregation=aggregation),
#     f"e_graphsage_{aggregation}_no_sampling": EGRAPHSAGE(ndim, edim, ndim_out, num_layers, activation, dropout,
#                                                          residual, num_classes, num_neighbors=None, aggregation=aggregation),
#     # "e_gat_sampling": EGAT(ndim, edim, ndim_out, num_layers, activation, dropout,
#     #               residual, num_classes, num_neighbors=number_neighbors),
#     # "e_gat_no_sampling": EGAT(ndim, edim, ndim_out, num_layers, activation, dropout, residual, num_classes, num_neighbors=None),
# }

# criterion = nn.CrossEntropyLoss(data_module.train_dataset.class_weights)

# for model_name, model in my_models.items():
        
#     config = {
#         "run_dtime": run_dtime,
#         "type": "GNN",
#         "model_name": model_name,
#         "max_epochs": max_epochs,
#         "learning_rate": learning_rate,
#         "weight_decay": weight_decay,
#         "ndim_out": ndim_out,
#         "num_layers": num_layers,
#         "number_neighbors": number_neighbors,
#         "activation": activation.__name__,
#         "dropout": dropout,
#         "residual": residual,
#         "multi_class": multi_class,
#         "aggregation": aggregation,
#         "early_stopping_patience": early_stopping_patience,
#         "use_centralities_nfeats": use_centralities_nfeats,}

#     graph_model = GraphModel(model, criterion, learning_rate, config, model_name,
#                              labels_mapping, weight_decay=weight_decay, using_wandb=using_wandb, norm=False, multi_class=True)

    
#     if using_wandb:
#         wandb_logger = WandbLogger(
#             project=f"GNN-Analysis-{dataset.name}", # Project Name Wandb
#             name=f"{model_name}---{g_type}",
#             config=config,
#             save_dir=wandb_runs_path
#         )
#     else:
#         wandb_logger = None
        
#     f1_checkpoint_callback = ModelCheckpoint(
#         monitor="val_f1_score",
#         mode="max",
#         filename="best-val-f1-{epoch:02d}-{val_f1_score:.2f}",
#         save_top_k=save_top_k,
#         verbose=False,
#     )
#     early_stopping_callback = EarlyStopping(
#         monitor="val_loss",
#         mode="min",
#         patience=early_stopping_patience,
#         verbose=False,
#     )

#     trainer = pl.Trainer(
#         max_epochs=max_epochs,
#         num_sanity_val_steps=0,
#         log_every_n_steps=0,
#         callbacks=[
#             f1_checkpoint_callback,
#             early_stopping_callback
#         ],
#         default_root_dir=logs_folder,
#         logger=wandb_logger if using_wandb else None,
#     )

#     trainer.fit(graph_model, datamodule=data_module)
    
#     test_results = []
    
#     for i, k in enumerate(f1_checkpoint_callback.best_k_models.keys()):
#         graph_model.test_prefix = f"best_f1_{i}"
#         results = trainer.test(
#             graph_model, datamodule=data_module, ckpt_path=k)
#         test_results.append(results[0][f"best_f1_{i}_test_f1"])

#     logs = {
#         "median_f1_of_best_f1": np.median(test_results),
#         "max_f1_of_best_f1": np.max(test_results),
#         "avg_f1_of_best_f1": np.mean(test_results)
#     }

#     if using_wandb:
#         wandb.log(logs)
#         wandb.finish()
#     else:
#         trainer.logger.log_metrics(logs, step=trainer.global_step)

In [ ]:
 # wandb.finish()

<h2>EGraphSAGE Sampling mean</h2>

In [ ]:
# print("----------------------- TRAINING EGraphSAGE Sampling mean --------------------------")

In [ ]:
# warnings.filterwarnings("ignore", ".*does not have many workers.*")

# dataset = datasets[dataset_name]

# # Hyperparameters
# aggregation="mean"

# activation = F.relu
# graphs_folder = graph_folder
# if dataset_name == "cic_ids_2017":
#     graphs_folder = "/kaggle/working/cic_ids_2017/flow__multi_class__n_feats__unsorted"
# logs_folder = os.path.join("logs", dataset.name)
# os.makedirs(logs_folder, exist_ok=True)
# wandb_runs_path = os.path.join("logs", "wandb_runs")
# os.makedirs(wandb_runs_path, exist_ok=True)

# labels_mapping = {0: "Normal", 1: "Attack"}
# num_classes = 2
# if multi_class:
#     with open(os.path.join(dataset_folder, "labels_names.pkl"), "rb") as f:
#     # /kaggle/input/cic-ids-2017-gnn-2/labels_names.pkl
#     # with open("/kaggle/input/cic-ids-2017-gnn-2/labels_names.pkl", "rb") as f:
#         labels_names = pickle.load(f)
#     labels_mapping = labels_names[0]
# num_classes = len(labels_mapping)

# dataset_kwargs = dict(
#     use_node_features=use_centralities_nfeats,
#     multi_class=True,
#     using_masking=False,
#     masked_class=2,
#     num_workers=0,
#     label_col=dataset.label_col,
#     class_num_col=dataset.class_num_col,
#     device='cuda' if torch.cuda.is_available() else "cpu"
# )

# data_module = GraphDataModule(
#     graphs_folder, batch_size=1, **dataset_kwargs)
# data_module.setup()

# ndim = next(iter(data_module.train_dataloader())).ndata["h"].shape[-1]
# edim = next(iter(data_module.train_dataloader())).edata['h'].shape[-1]

# my_models = {
#     # "e_gcn": EGCN(ndim, edim, ndim_out, num_layers, activation,
#     #               dropout, residual, num_classes),
#     f"e_graphsage_{aggregation}": EGRAPHSAGE(ndim, edim, ndim_out, num_layers, activation, dropout,
#                                              residual, num_classes, num_neighbors=number_neighbors, aggregation=aggregation),
#     # f"e_graphsage_{aggregation}_no_sampling": EGRAPHSAGE(ndim, edim, ndim_out, num_layers, activation, dropout,
#     #                                                      residual, num_classes, num_neighbors=None, aggregation=aggregation),
#     # "e_gat_sampling": EGAT(ndim, edim, ndim_out, num_layers, activation, dropout,
#                   # residual, num_classes, num_neighbors=number_neighbors),
#     # "e_gat_no_sampling": EGAT(ndim, edim, ndim_out, num_layers, activation, dropout, residual, num_classes, num_neighbors=None),
# }

# criterion = nn.CrossEntropyLoss(data_module.train_dataset.class_weights)

# for model_name, model in my_models.items():
        
#     config = {
#         "run_dtime": run_dtime,
#         "type": "GNN",
#         "model_name": model_name,
#         "max_epochs": max_epochs,
#         "learning_rate": learning_rate,
#         "weight_decay": weight_decay,
#         "ndim_out": ndim_out,
#         "num_layers": num_layers,
#         "number_neighbors": number_neighbors,
#         "activation": activation.__name__,
#         "dropout": dropout,
#         "residual": residual,
#         "multi_class": multi_class,
#         "aggregation": aggregation,
#         "early_stopping_patience": early_stopping_patience,
#         "use_centralities_nfeats": use_centralities_nfeats,}

#     graph_model = GraphModel(model, criterion, learning_rate, config, model_name,
#                              labels_mapping, weight_decay=weight_decay, using_wandb=using_wandb, norm=False, multi_class=True)

    
#     if using_wandb:
#         wandb_logger = WandbLogger(
#             project=f"GNN-Analysis-{dataset.name}", # Project Name Wandb
#             name=f"{model_name}---{g_type}",
#             config=config,
#             save_dir=wandb_runs_path
#         )
#     else:
#         wandb_logger = None
        
#     f1_checkpoint_callback = ModelCheckpoint(
#         monitor="val_f1_score",
#         mode="max",
#         filename="best-val-f1-{epoch:02d}-{val_f1_score:.2f}",
#         save_top_k=save_top_k,
#         verbose=False,
#     )
#     early_stopping_callback = EarlyStopping(
#         monitor="val_loss",
#         mode="min",
#         patience=early_stopping_patience,
#         verbose=False,
#     )

#     trainer = pl.Trainer(
#         max_epochs=max_epochs,
#         num_sanity_val_steps=0,
#         log_every_n_steps=0,
#         callbacks=[
#             f1_checkpoint_callback,
#             early_stopping_callback
#         ],
#         default_root_dir=logs_folder,
#         logger=wandb_logger if using_wandb else None,
#     )

#     trainer.fit(graph_model, datamodule=data_module)
    
#     test_results = []
    
#     for i, k in enumerate(f1_checkpoint_callback.best_k_models.keys()):
#         graph_model.test_prefix = f"best_f1_{i}"
#         results = trainer.test(
#             graph_model, datamodule=data_module, ckpt_path=k)
#         test_results.append(results[0][f"best_f1_{i}_test_f1"])

#     logs = {
#         "median_f1_of_best_f1": np.median(test_results),
#         "max_f1_of_best_f1": np.max(test_results),
#         "avg_f1_of_best_f1": np.mean(test_results)
#     }

#     if using_wandb:
#         wandb.log(logs)
#         wandb.finish()
#     else:
#         trainer.logger.log_metrics(logs, step=trainer.global_step)

<h2>EGraphSAGE Sampling sum</h2>

In [ ]:
# print("----------------------- TRAINING EGraphSAGE Sampling sum --------------------------")

In [ ]:
# warnings.filterwarnings("ignore", ".*does not have many workers.*")

# dataset = datasets[dataset_name]

# # Hyperparameters
# aggregation="sum"

# activation = F.relu
# graphs_folder = graph_folder
# if dataset_name == "cic_ids_2017":
#     graphs_folder = "/kaggle/working/cic_ids_2017/flow__multi_class__n_feats__unsorted"
# logs_folder = os.path.join("logs", dataset.name)
# os.makedirs(logs_folder, exist_ok=True)
# wandb_runs_path = os.path.join("logs", "wandb_runs")
# os.makedirs(wandb_runs_path, exist_ok=True)

# labels_mapping = {0: "Normal", 1: "Attack"}
# num_classes = 2
# if multi_class:
#     with open(os.path.join(dataset_folder, "labels_names.pkl"), "rb") as f:
#     # /kaggle/input/cic-ids-2017-gnn-2/labels_names.pkl
#     # with open("/kaggle/input/cic-ids-2017-gnn-2/labels_names.pkl", "rb") as f:
#         labels_names = pickle.load(f)
#     labels_mapping = labels_names[0]
# num_classes = len(labels_mapping)

# dataset_kwargs = dict(
#     use_node_features=use_centralities_nfeats,
#     multi_class=True,
#     using_masking=False,
#     masked_class=2,
#     num_workers=0,
#     label_col=dataset.label_col,
#     class_num_col=dataset.class_num_col,
#     device='cuda' if torch.cuda.is_available() else "cpu"
# )

# data_module = GraphDataModule(
#     graphs_folder, batch_size=1, **dataset_kwargs)
# data_module.setup()

# ndim = next(iter(data_module.train_dataloader())).ndata["h"].shape[-1]
# edim = next(iter(data_module.train_dataloader())).edata['h'].shape[-1]

# my_models = {
#     # "e_gcn": EGCN(ndim, edim, ndim_out, num_layers, activation,
#     #               dropout, residual, num_classes),
#     f"e_graphsage_{aggregation}": EGRAPHSAGE(ndim, edim, ndim_out, num_layers, activation, dropout,
#                                              residual, num_classes, num_neighbors=number_neighbors, aggregation=aggregation),
#     # f"e_graphsage_{aggregation}_no_sampling": EGRAPHSAGE(ndim, edim, ndim_out, num_layers, activation, dropout,
#     #                                                      residual, num_classes, num_neighbors=None, aggregation=aggregation),
#     # "e_gat_sampling": EGAT(ndim, edim, ndim_out, num_layers, activation, dropout,
#                   # residual, num_classes, num_neighbors=number_neighbors),
#     # "e_gat_no_sampling": EGAT(ndim, edim, ndim_out, num_layers, activation, dropout, residual, num_classes, num_neighbors=None),
# }

# criterion = nn.CrossEntropyLoss(data_module.train_dataset.class_weights)

# for model_name, model in my_models.items():
        
#     config = {
#         "run_dtime": run_dtime,
#         "type": "GNN",
#         "model_name": model_name,
#         "max_epochs": max_epochs,
#         "learning_rate": learning_rate,
#         "weight_decay": weight_decay,
#         "ndim_out": ndim_out,
#         "num_layers": num_layers,
#         "number_neighbors": number_neighbors,
#         "activation": activation.__name__,
#         "dropout": dropout,
#         "residual": residual,
#         "multi_class": multi_class,
#         "aggregation": aggregation,
#         "early_stopping_patience": early_stopping_patience,
#         "use_centralities_nfeats": use_centralities_nfeats,}

#     graph_model = GraphModel(model, criterion, learning_rate, config, model_name,
#                              labels_mapping, weight_decay=weight_decay, using_wandb=using_wandb, norm=False, multi_class=True)

    
#     if using_wandb:
#         wandb_logger = WandbLogger(
#             project=f"GNN-Analysis-{dataset.name}", # Project Name Wandb
#             name=f"{model_name}---{g_type}",
#             config=config,
#             save_dir=wandb_runs_path
#         )
#     else:
#         wandb_logger = None
        
#     f1_checkpoint_callback = ModelCheckpoint(
#         monitor="val_f1_score",
#         mode="max",
#         filename="best-val-f1-{epoch:02d}-{val_f1_score:.2f}",
#         save_top_k=save_top_k,
#         verbose=False,
#     )
#     early_stopping_callback = EarlyStopping(
#         monitor="val_loss",
#         mode="min",
#         patience=early_stopping_patience,
#         verbose=False,
#     )

#     trainer = pl.Trainer(
#         max_epochs=max_epochs,
#         num_sanity_val_steps=0,
#         log_every_n_steps=0,
#         callbacks=[
#             f1_checkpoint_callback,
#             early_stopping_callback
#         ],
#         default_root_dir=logs_folder,
#         logger=wandb_logger if using_wandb else None,
#     )

#     trainer.fit(graph_model, datamodule=data_module)
    
#     test_results = []
    
#     for i, k in enumerate(f1_checkpoint_callback.best_k_models.keys()):
#         graph_model.test_prefix = f"best_f1_{i}"
#         results = trainer.test(
#             graph_model, datamodule=data_module, ckpt_path=k)
#         test_results.append(results[0][f"best_f1_{i}_test_f1"])

#     logs = {
#         "median_f1_of_best_f1": np.median(test_results),
#         "max_f1_of_best_f1": np.max(test_results),
#         "avg_f1_of_best_f1": np.mean(test_results)
#     }

#     if using_wandb:
#         wandb.log(logs)
#         wandb.finish()
#     else:
#         trainer.logger.log_metrics(logs, step=trainer.global_step)

<h2>E_GCN</h2>

In [ ]:
# print("----------------------- TRAINING E_GCN --------------------------")

In [ ]:
# warnings.filterwarnings("ignore", ".*does not have many workers.*")

# dataset = datasets[dataset_name]

# # Hyperparameters
# aggregation="mean"

# activation = F.relu
# graphs_folder = graph_folder
# if dataset_name == "cic_ids_2017":
#     graphs_folder = "/kaggle/working/cic_ids_2017/flow__multi_class__n_feats__unsorted"
# logs_folder = os.path.join("logs", dataset.name)
# os.makedirs(logs_folder, exist_ok=True)
# wandb_runs_path = os.path.join("logs", "wandb_runs")
# os.makedirs(wandb_runs_path, exist_ok=True)

# labels_mapping = {0: "Normal", 1: "Attack"}
# num_classes = 2
# if multi_class:
#     with open(os.path.join(dataset_folder, "labels_names.pkl"), "rb") as f:
#     # /kaggle/input/cic-ids-2017-gnn-2/labels_names.pkl
#     # with open("/kaggle/input/cic-ids-2017-gnn-2/labels_names.pkl", "rb") as f:
#         labels_names = pickle.load(f)
#     labels_mapping = labels_names[0]
# num_classes = len(labels_mapping)

# dataset_kwargs = dict(
#     use_node_features=use_centralities_nfeats,
#     multi_class=True,
#     using_masking=False,
#     masked_class=2,
#     num_workers=0,
#     label_col=dataset.label_col,
#     class_num_col=dataset.class_num_col,
#     device='cuda' if torch.cuda.is_available() else "cpu"
# )

# data_module = GraphDataModule(
#     graphs_folder, batch_size=1, **dataset_kwargs)
# data_module.setup()

# ndim = next(iter(data_module.train_dataloader())).ndata["h"].shape[-1]
# edim = next(iter(data_module.train_dataloader())).edata['h'].shape[-1]

# my_models = {
#     "e_gcn": EGCN(ndim, edim, ndim_out, num_layers, activation,
#                   dropout, residual, num_classes),
#     # f"e_graphsage_{aggregation}": EGRAPHSAGE(ndim, edim, ndim_out, num_layers, activation, dropout,
#     #                                          residual, num_classes, num_neighbors=number_neighbors, aggregation=aggregation),
#     # f"e_graphsage_{aggregation}_no_sampling": EGRAPHSAGE(ndim, edim, ndim_out, num_layers, activation, dropout,
#     #                                                      residual, num_classes, num_neighbors=None, aggregation=aggregation),
#     # "e_gat_sampling": EGAT(ndim, edim, ndim_out, num_layers, activation, dropout,
#                   # residual, num_classes, num_neighbors=number_neighbors),
#     # "e_gat_no_sampling": EGAT(ndim, edim, ndim_out, num_layers, activation, dropout, residual, num_classes, num_neighbors=None),
# }

# criterion = nn.CrossEntropyLoss(data_module.train_dataset.class_weights)

# for model_name, model in my_models.items():
        
#     config = {
#         "run_dtime": run_dtime,
#         "type": "GNN",
#         "model_name": model_name,
#         "max_epochs": max_epochs,
#         "learning_rate": learning_rate,
#         "weight_decay": weight_decay,
#         "ndim_out": ndim_out,
#         "num_layers": num_layers,
#         "number_neighbors": number_neighbors,
#         "activation": activation.__name__,
#         "dropout": dropout,
#         "residual": residual,
#         "multi_class": multi_class,
#         "aggregation": aggregation,
#         "early_stopping_patience": early_stopping_patience,
#         "use_centralities_nfeats": use_centralities_nfeats,}

#     graph_model = GraphModel(model, criterion, learning_rate, config, model_name,
#                              labels_mapping, weight_decay=weight_decay, using_wandb=using_wandb, norm=False, multi_class=True)

    
#     if using_wandb:
#         wandb_logger = WandbLogger(
#             project=f"GNN-Analysis-{dataset.name}", # Project Name Wandb
#             name=f"{model_name}---{g_type}",
#             config=config,
#             save_dir=wandb_runs_path
#         )
#     else:
#         wandb_logger = None
        
#     f1_checkpoint_callback = ModelCheckpoint(
#         monitor="val_f1_score",
#         mode="max",
#         filename="best-val-f1-{epoch:02d}-{val_f1_score:.2f}",
#         save_top_k=save_top_k,
#         verbose=False,
#     )
#     early_stopping_callback = EarlyStopping(
#         monitor="val_loss",
#         mode="min",
#         patience=early_stopping_patience,
#         verbose=False,
#     )

#     trainer = pl.Trainer(
#         max_epochs=max_epochs,
#         num_sanity_val_steps=0,
#         log_every_n_steps=0,
#         callbacks=[
#             f1_checkpoint_callback,
#             early_stopping_callback
#         ],
#         default_root_dir=logs_folder,
#         logger=wandb_logger if using_wandb else None,
#     )

#     trainer.fit(graph_model, datamodule=data_module)
    
#     test_results = []
    
#     for i, k in enumerate(f1_checkpoint_callback.best_k_models.keys()):
#         graph_model.test_prefix = f"best_f1_{i}"
#         results = trainer.test(
#             graph_model, datamodule=data_module, ckpt_path=k)
#         test_results.append(results[0][f"best_f1_{i}_test_f1"])

#     logs = {
#         "median_f1_of_best_f1": np.median(test_results),
#         "max_f1_of_best_f1": np.max(test_results),
#         "avg_f1_of_best_f1": np.mean(test_results)
#     }

#     if using_wandb:
#         wandb.log(logs)
#         wandb.finish()
#     else:
#         trainer.logger.log_metrics(logs, step=trainer.global_step)